In [1]:
import os
import orjson
import pandas as pd
import torch
from torchvision.ops import box_convert
import cv2

In [6]:
json_path = "dataset/test/result_manyaug/bbox.json"
conf_thresh = 0.7
img_width, img_height = 1280, 720

with open(json_path, "rb") as f:
    json_data = orjson.loads(f.read())

results_dict = {}
for result in json_data:
    if result["score"] < conf_thresh:
        continue

    if result["image_id"] not in results_dict:
        results_dict[result["image_id"]] = []

    x, y, w, h = result["bbox"]
    
    results_dict[result["image_id"]].append({
        "Image_ID": "image_" + str(result["image_id"]).zfill(4),
        "class": 0,
        "confidence": result["score"],
        "ymin": y / img_height,
        "xmin": x / img_width,
        "ymax": (y + h) / img_height,
        "xmax": (x + w) / img_width,
    })

results_list = []
for img_id in results_dict:
    results_dict[img_id].sort(key=lambda result: result["confidence"], reverse=True)
    results_list.extend(results_dict[img_id][:4])

# ensure that every image has at least 1 bbox
# required by DSTA
assert len(set([result["Image_ID"] for result in results_list])) == 1600 # hardcoded for now

results_list.sort(key=lambda result: result["Image_ID"])

results_df = pd.DataFrame(results_list)
results_df.to_csv('submissions/0.882 manyaug best conf 0.7 no reid.csv', index=False)

In [7]:
results_df_old = results_df
len(results_df_old)

3409

In [3]:
a = sorted(list(set([result["Image_ID"] for result in results_list])))

In [4]:
all_images = [f'image_{str(i).zfill(4)}' for i in range(1600)]
for i in a:
    all_images.remove(i)

print(all_images)

[]
